# Import required package for feature engineering

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# For mapping to Google Drive
from google.colab import drive

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/lohdaijiu/BT4012/main/fraud_oracle.csv")

In [ ]:
df.columns

Index(['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea',
       'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex',
       'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory',
       'VehiclePrice', 'FraudFound_P', 'PolicyNumber', 'RepNumber',
       'Deductible', 'DriverRating', 'Days_Policy_Accident',
       'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle',
       'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType',
       'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars', 'Year',
       'BasePolicy'],
      dtype='object')

In [ ]:
df

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15415,Nov,4,Friday,Toyota,Urban,Tuesday,Nov,5,Male,Married,...,6 years,31 to 35,No,No,External,none,no change,1 vehicle,1996,Collision
15416,Nov,5,Thursday,Pontiac,Urban,Friday,Dec,1,Male,Married,...,6 years,31 to 35,No,No,External,more than 5,no change,3 to 4,1996,Liability
15417,Nov,5,Thursday,Toyota,Rural,Friday,Dec,1,Male,Single,...,5 years,26 to 30,No,No,External,1 to 2,no change,1 vehicle,1996,Collision
15418,Dec,1,Monday,Toyota,Urban,Thursday,Dec,2,Female,Married,...,2 years,31 to 35,No,No,External,more than 5,no change,1 vehicle,1996,All Perils


In [ ]:
categorical_variables = ["Make", "AccidentArea", "Sex", "MaritalStatus", "Fault", "PolicyType", "VehicleCategory",  "AgentType", "WitnessPresent", "PoliceReportFiled", "RepNumber", "BasePolicy"]
ordinal_variables = ["VehiclePrice", "AgeOfPolicyHolder", "Deductible", "DriverRating", "PastNumberOfClaims", "AgeOfVehicle", "AgeOfPolicyHolder", "AddressChange_Claim", "NumberOfSuppliments", "NumberOfCars"]
numerical_variables = ["Age"]
time_variables = ["Month", "Week of Month", "DayOfWeek", "MonthClaimed", "DayOfWeekClaimed", "WeekOfMonthClaimed"]

## Removal of invalid data points from EDA

In [ ]:
df = df.loc[df["MonthClaimed"] != '0']

## Drop irrelevant/unuseful features

In [ ]:
df = df.drop(columns = ['Year', 'Days_Policy_Accident', 'Days_Policy_Claim', "PolicyNumber", "Age"])


# Feature Engineering for categorical variables

In [ ]:
#df = pd.get_dummies(df, columns=categorical_variables)

### Make variable


In [ ]:
df['Make'].value_counts()

Pontiac      3837
Toyota       3121
Honda        2800
Mazda        2354
Chevrolet    1681
Accura        472
Ford          450
VW            283
Dodge         109
Saab          108
Mercury        83
Saturn         58
Nisson         30
BMW            15
Jaguar          6
Porche          5
Mecedes         4
Ferrari         2
Lexus           1
Name: Make, dtype: int64

Collate variables that occur less than 50 times into Others

In [ ]:
def collate_make(x):
    if x in ['Nisson', 'BMW', 'Jaguar', 'Porche', 'Mecedes', 'Ferrari', 'Lexus']:
        return "Others"
    else:
        return x

df['Make'] = df['Make'].apply(lambda x: collate_make(x))
print(df['Make'].value_counts())

Pontiac      3837
Toyota       3121
Honda        2800
Mazda        2354
Chevrolet    1681
Accura        472
Ford          450
VW            283
Dodge         109
Saab          108
Mercury        83
Others         63
Saturn         58
Name: Make, dtype: int64


In [ ]:
df = pd.get_dummies(df, columns=['Make'])

### Marital Status Variable

In [ ]:
df['MaritalStatus'].value_counts()

Married     10625
Single       4683
Divorced       76
Widow          35
Name: MaritalStatus, dtype: int64

In [ ]:
def collate_marital_status(x):
    if x in ['Divorced', 'Widow']:
        return "Others"
    else:
        return x

df['MaritalStatus'] = df['MaritalStatus'].apply(lambda x: collate_marital_status(x))
print(df['MaritalStatus'].value_counts())

Married    10625
Single      4683
Others       111
Name: MaritalStatus, dtype: int64


In [ ]:
df = pd.get_dummies(df, columns=['MaritalStatus'])

### Policy Type variable

In [ ]:
df['PolicyType'].value_counts()

Sedan - Collision       5584
Sedan - Liability       4987
Sedan - All Perils      4086
Sport - Collision        348
Utility - All Perils     340
Utility - Collision       30
Sport - All Perils        22
Utility - Liability       21
Sport - Liability          1
Name: PolicyType, dtype: int64

In [ ]:
def collate_policy_type(x):
    if x in ['Utility - Collision', 'Sport - All Perils', 'Utility - Liability', 'Sport - Liability']:
        return "Others"
    else:
        return x

df['PolicyType'] = df['PolicyType'].apply(lambda x: collate_policy_type(x))
print(df['PolicyType'].value_counts())

Sedan - Collision       5584
Sedan - Liability       4987
Sedan - All Perils      4086
Sport - Collision        348
Utility - All Perils     340
Others                    74
Name: PolicyType, dtype: int64


In [ ]:
df = pd.get_dummies(df, columns=['PolicyType'])

### Creating dummy variables for the rest of the categorical variables

In [ ]:
remaining_categorical = ["AccidentArea", "Sex",  "Fault", "VehicleCategory",  "AgentType", "WitnessPresent", "PoliceReportFiled", "RepNumber", "BasePolicy"]
df = pd.get_dummies(df, columns=remaining_categorical)

#Feature Engineering for Ordinal Variables

In [ ]:
print(df["VehiclePrice"].value_counts())


20000 to 29000     8079
30000 to 39000     3533
more than 69000    2163
less than 20000    1096
40000 to 59000      461
60000 to 69000       87
Name: VehiclePrice, dtype: int64


In [ ]:
price = ["less than 20000", "20000 to 29000", "30000 to 39000", "40000 to 59000", "60000 to 69000", "more than 69000"]
enc = OrdinalEncoder(categories = [price])

df["VehiclePrice_categories"] = enc.fit_transform(df.loc[:,['VehiclePrice']])

In [ ]:
age = ['16 to 17', '18 to 20', '21 to 25', '26 to 30','31 to 35','36 to 40','41 to 50','51 to 65','over 65',]
enc = OrdinalEncoder(categories = [age])
df["AgeOfPolicyHolder_categories"] = enc.fit_transform(df.loc[:,['AgeOfPolicyHolder']])

In [ ]:
deductible = [300, 400, 500, 700]
enc = OrdinalEncoder(categories = [deductible])
df["Deductible_categories"] = enc.fit_transform(df.loc[:,['Deductible']])

In [ ]:
df["PastNumberOfClaims"].value_counts()
number = ["none", "1", "2 to 4", "more than 4"]
enc = OrdinalEncoder(categories = [number])
df['PastNumberOfClaims_categories'] = enc.fit_transform(df.loc[:,['PastNumberOfClaims']])

In [ ]:
df["AgeOfVehicle"].value_counts()
vehicle_age = ["new", "2 years", "3 years", "4 years", "5 years", "6 years", "7 years", "more than 7"]
enc = OrdinalEncoder(categories = [vehicle_age])
df['AgeOfVehicle_categories'] = enc.fit_transform(df.loc[:,['AgeOfVehicle']])

In [ ]:
address_change = ["no change", 'under 6 months', '1 year', "2 to 3 years", "4 to 8 years"]
enc = OrdinalEncoder(categories = [address_change])
df['AddressChange_Claim_categories'] = enc.fit_transform(df.loc[:,['AddressChange_Claim']])

In [ ]:
df["NumberOfSuppliments"].value_counts()
suppliments = ["none", "1 to 2", "3 to 5", "more than 5"]
enc = OrdinalEncoder(categories = [suppliments])
df['NumberOfSuppliments_categories'] = enc.fit_transform(df.loc[:,['NumberOfSuppliments']])

In [ ]:
cars = ["1 vehicle", "2 vehicles", '3 to 4', '5 to 8', 'more than 8']
enc = OrdinalEncoder(categories = [cars])
df['NumberOfCars_categories'] = enc.fit_transform(df.loc[:,['NumberOfCars']])

# Remove string ordinal variables

In [ ]:
df.drop(columns = ordinal_variables, inplace = True)

# Time variable feature engineering using cyclical encoding

In [ ]:
#Cyclical Encoding for time variables
days = {"Monday":1, "Tuesday":2, "Wednesday":3, "Thursday":4, "Friday":5, "Saturday":6, "Sunday":7}
df['DayOfWeek_numeric'] = df['DayOfWeek'].apply(lambda x: days.get(x))
df['DayOfWeekClaimed_numeric'] = df['DayOfWeekClaimed'].apply(lambda x: days.get(x))
months = {"Jan":1, "Feb":2, "Mar":3, "Apr": 4, "May":5, "Jun":6, "Jul": 7, "Aug":8, "Sep":9, "Oct":10, "Nov":11, "Dec":12}
df['MonthClaimed_numeric'] = df['MonthClaimed'].apply(lambda x: months.get(x))
df['Month_numeric'] = df['Month'].apply(lambda x: months.get(x))

In [ ]:
def cyclical_encoding(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [ ]:
time_variables = ["Month_numeric", "WeekOfMonth", "DayOfWeek_numeric", "MonthClaimed_numeric", "DayOfWeekClaimed_numeric", "WeekOfMonthClaimed"]

for column in time_variables:
    max_values = [12, 5, 7, 12, 5, 7]
    pointer = 0
    cyclical_encoding(df, column, max_values[pointer])
    pointer += 1

In [ ]:
df.drop(columns = time_variables, inplace = True)

In [ ]:
df.drop(columns = ["Month", "DayOfWeek", "DayOfWeekClaimed", "MonthClaimed"], inplace = True)

In [ ]:
df.to_csv("Model_Data.csv")

In [ ]:
df.isna().any()

FraudFound_P                    False
Make_Accura                     False
Make_Chevrolet                  False
Make_Dodge                      False
Make_Ford                       False
                                ...  
MonthClaimed_numeric_cos        False
DayOfWeekClaimed_numeric_sin    False
DayOfWeekClaimed_numeric_cos    False
WeekOfMonthClaimed_sin          False
WeekOfMonthClaimed_cos          False
Length: 77, dtype: bool